In [1]:
import torch
import model
import torch.nn as nn
import tifffile as tif
import utils

# 加载模型

In [2]:
# 载入模型
generator = model.DCGAN3D_G_CPU(isize=64, nz=512, nc=1, ngf=64, ngpu=0)

# 加载训练好的模型权重
checkpoint = torch.load('./model/berea_64/netG_epoch_700.pth', map_location='cpu')

checkpoint = {k: v for k, v in checkpoint.items() if 'phase_low' not in k and 'phase_high' not in k}
generator.load_state_dict(checkpoint, strict=False)

RuntimeError: Error(s) in loading state_dict for DCGAN3D_G_CPU:
	size mismatch for main.6.weight: copying a param with shape torch.Size([256, 128, 4, 4, 4]) from checkpoint, the shape in current model is torch.Size([64, 1, 4, 4, 4]).

# 读数据

In [6]:
data = tif.imread("./data/berea_10_1.tif")
# 将数据转换为 float32 类型
data_tensor = torch.tensor(data, dtype=torch.float32)

# 扩展维度为 (1, 1, 64, 64)
data_tensor = torch.unsqueeze(torch.unsqueeze(data_tensor, 0), 0)
data_tensor.shape

torch.Size([1, 1, 128, 128])

# 编码器编码

In [7]:
# 假设我们想要的输出大小是 [n, 512, 5, 5, 5]
b = 5
encoder = utils.Encoder(b, data_tensor.shape[-1])

# 通过编码器生成输出
output = encoder(data_tensor)

print(output.shape)  # 输出形状应为 [8, 512, 5, 5, 5]

torch.Size([1, 512, 5, 5, 5])


# 生成并保存

In [8]:
# 测试生成器 1 是 64  3是96 5 是128  13是256
# 输入大小为 [batch, nz, 1, 1, 1]
generated_output = generator(output)
print(generated_output.shape) 

torch.Size([1, 1, 128, 128, 128])


In [1]:
utils.save_tiff(generated_output, "./result/result_128.tif")

NameError: name 'utils' is not defined